In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re

from scipy.sparse import csr_matrix, coo_matrix

from implicit.nearest_neighbours import (ItemItemRecommender, CosineRecommender, 
                                         TFIDFRecommender, BM25Recommender)

from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямо пропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """
    Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    weights = items_weights['weight'].to_list()
    
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=weights)

    return recs.tolist()

In [5]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights.columns = ['item_id', 'weight']

items_weights = items_weights[items_weights['weight'] > 1]
items_weights['weight'] = np.log(items_weights['weight'])

items_weights['weight'] = items_weights['weight'] / items_weights['weight'].sum()

items_weights.head()

CPU times: user 63.5 ms, sys: 11.3 ms, total: 74.8 ms
Wall time: 75.1 ms


,item_id,weight
0,25671,0.000013
2,26093,0.000002
3,26190,0.000002
4,26355,0.000003
5,26426,0.000004


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [6]:
recommendations = pd.read_csv('predictions_basic.csv')
recommendations.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[9655242, 1355234, 953312, 177996, 9296877]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[13039379, 834616, 637374, 13672001, 821025]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 981760, 1098066, 826249, 999999]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[856099, 12188215, 9926646, 1097524, 2633621]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[1082185, 981760, 1127831, 999999, 878996]"
3,7,[ 840386 889774 898068 909714 929067 ...,"[3936316, 16123470, 927554, 1738034, 9526381]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[1082185, 981760, 1127831, 999999, 961554]"
4,8,[ 835098 872137 910439 924610 992977 ...,"[912806, 649441, 1811336, 1938253, 1098874]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]"


In [7]:
def precision_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list = bought_list
    recommended_list = recommended_list[:k]

    flags = np.isin(recommended_list, bought_list)

    precision = flags.sum() / len(recommended_list)

    return precision

In [8]:
for lst in recommendations.columns[1:]:
    recommendations.loc[:, lst] = recommendations[lst].map(
        lambda x: re.findall(r'[0-9][0-9]*', x[1:-1])).apply(lambda x: list(map(int, x)))

In [9]:
rec_list = ['random_recommendation', 'popular_recommendation', 
            'itemitem', 'cosine', 'tfidf', 'own_purchases']
comparison_df = pd.DataFrame(columns=['algorithm', 'precision@5'])

In [10]:
for a in recommendations.columns[2:]:
    metrics = recommendations.apply(lambda row: precision_at_k(row[a], row['actual']), axis=1)
    comparison_df.loc[len(comparison_df.index)] = [a, metrics.mean()]
comparison_df

,algorithm,precision@5
0,random_recommendation,0.000686
1,popular_recommendation,0.155240
2,itemitem,0.136925
3,cosine,0.132909
4,tfidf,0.138981
5,own_purchases,0.138981


Лучшее качество по метрике precision@5 показывает алгоритм popular_recommendation (впрочем, itemitem, cosine и tf-idf мы тоже рекомендовали на выборке 5К популярных товаров).